#Automatizando el proceso de extracción de datos

Hasta ahora hemos aprendido a obtener data de la web, extraerla e imprimirla. Sin embargo, por el momento nuestro acercamiento ha sido "estático": encontramos la página web que queremos, ubicamos aquello que queremos y lo extraemos. `Python` puede ir un poco más allá de esos simples pasos. Su verdadera fortaleza se demuestra cuando tenemos que obtener una serie de datos para los cuales se tienen que seguir los mismos procedimientos. 

Imaginen el siguiente ejemplo: tienen una lista de candidatos. Ustedes desean saber cuáles son los antecedentes de esos candidatos. Una manera sería ingresar a la hoja de vida de cada candidato y extraer aquello que se está buscando. Esto sería posible, sin embargo no eficiente. ¿Qué pasaría si quieren obtener las hojas de vida de TODOS los candidatos a nivel nacional? 

Para eso podemos servirnos de lo que sabemos hasta ahora para automatizar nuestro procedimiento de extracción de datos. En esta clase aprenderemos:

* Trabajar haciendo web crawling.
* Mecanizar la extracción de data.
* Guardar la data obtenida en formato CSV.


###Noten que...

Los métodos que aprenderemos hoy tienen limitaciones. Hay páginas web más complejas que usan `javascript` para modificar su contenido. En estos casos los métodos de hoy no serán suficientes para automatizar el proceso de extracción. Existen otras librerías de `Python` que te pueden ayudar en esa tarea. Si te enfrentas a una página web compleja, te recomiendo que revises las posibilidades que te ofrece la librería `Selenium`. Revisa su documentación [aquí](https://selenium-python.readthedocs.org/).

## Regresando a nuestro ejemplo de Wikipedia!

Se acuerdan que como parte de la extracción de la tabla de partidos políticos obtuvimos una serie de links de sus respectivas páginas web de Wikipedia. Ahora veamos como sacar ciertos datos de manera automatizada de cada una de las páginas web de Wikipedia de los partidos políticos. 

Comencemos viendo cómo extraer estos datos de una de las páginas y evaluando si el mismo método funciona para las demás. Necesito tres datos de los partidos políticos:

1. Presidente.
2. Ideología.
3. Número de congresistas.

Aplica lo que has aprendido hasta ahora de BeatifulSoup y obtén estos tres datos por tu cuenta. Comencemos por la página web de Acción popular.

In [ ]:
import requests
from bs4 import BeautifulSoup

pag_accion_pop = requests.get("https://es.wikipedia.org/wiki/Acci%C3%B3n_Popular")
ap_obj = BeautifulSoup(pag_accion_pop.text, "html.parser")
presidente = None
ideología = None
congreso = None
for item in ap_obj.find("table", {"class": "infobox"}).findAll("th", {"scope":"row"}):
    if item.get_text() == "Presidente":
        presidente = item.find_next_sibling().get_text().strip()
    elif item.get_text() == "Ideología":
        ideología = item.find_next_sibling().get_text().strip()
    elif item.get_text() == "Congreso":
        congreso = item.find_next_sibling().get_text().strip()
print(presidente, ideología, congreso)
    


In [42]:
ap_obj.find("table", {"class": "infobox"}).findAll("th", {"scope":"row"})

[<th scope="row" style="text-align:left;">Presidente</th>,
 <th scope="row" style="text-align:left;">Vicepresidente</th>,
 <th scope="row" style="text-align:left;">Secretario/a general</th>,
 <th scope="row" style="text-align:left;">Fundación</th>,
 <th scope="row" style="text-align:left;">Ideología</th>,
 <th scope="row" style="text-align:left;">Posición en el espectro</th>,
 <th scope="row" style="text-align:left;">Sede</th>,
 <th scope="row" style="text-align:left;"><a href="/wiki/Congreso_de_la_Rep%C3%BAblica_del_Per%C3%BA" title="Congreso de la República del Perú">Congreso</a></th>,
 <th scope="row" style="text-align:left;">Sitio web</th>]

## ¿Funcionará para otro partido?

Probemos con el APRA. Modifiquemos nuestro código y veamos si cuando lo aplicamos para la página del APRA obtenemos la información que queremos.

In [ ]:
pag_apra = requests.get("https://es.wikipedia.org/wiki/Alianza_Popular_Revolucionaria_Americana")
apra_obj = BeautifulSoup(pag_apra.text, "html.parser")
for item in apra_obj.find("table", {"class": "infobox"}).findAll("th", {"scope":"row"}):
    if item.get_text() in ["Presidente", "Ideología", "Congreso"]:
        print(item.find_next_sibling().get_text().strip())

Hasta ahora todo parece estar funcionando a la perfección. Pero qué pasa cuando vamos a la página web del partido "Orden".


In [ ]:
pag_orden = requests.get("https://es.wikipedia.org/wiki/Ántero_Flores-Aráoz")
orden_obj = BeautifulSoup(pag_orden.text, "html.parser")
for item in orden_obj.find("table", {"class": "infobox"}).findAll("th", {"scope":"row"}):
    if item.get_text() in ["Presidente", "Ideología", "Congreso"]:
        print(item.find_next_sibling().get_text().strip())

No obtenemos ningún resultado. Esto no se debe a que nuestra extracción esté equivocada. Viendo la página web podemos darnos cuenta de que tiene otro formato a las anteriores. En efecto, esta no es la página web del partido, sino la de su fundador. Nuestro futuro script con el que extraeremos los datos debe tener en cuenta esto para poder incluir `missings` donde corresponde. 

## ¡Comencemos a automatizar nuestro proceso de extracción de datos!

Como se han dado cuenta, hemos podido reutilizar el código de extracción de datos y obtener los mismos resultados para diferentes páginas web. Esto nos indica que podemos ahorrar tiempo automatizando todo el proceso.

Lo primero que tenemos que hacer ahora es tratar de obtener todos los links de los partidos políticos. Esto no debe resultar muy difícil, ya que lo hicimos la vez pasada. 

In [43]:
# Obtén los links de las páginas web de Wikipedia de los partidos políticos
pag_wiki = requests.get("https://es.wikipedia.org/wiki/Partidos_pol%C3%ADticos_del_Per%C3%BA")
wiki_pag = BeautifulSoup(pag_wiki.text, "html.parser")

tabla_partidos = []
for item in wiki_pag.findAll("tr"):
    try:
        tabla_partidos.append(item.find_next_sibling().find("td").a.attrs['href']) ## Notar que aquí existen algunos None!
    except AttributeError:
        continue

print(tabla_partidos)

['/wiki/Acci%C3%B3n_Popular', '/wiki/Alianza_para_el_Progreso_(Per%C3%BA)', '/wiki/Fuerza_2011', '/wiki/%C3%81ntero_Flores-Ar%C3%A1oz', '/wiki/Partido_Aprista_Peruano', '/wiki/Partido_Democr%C3%A1tico_Somos_Per%C3%BA', '/wiki/Partido_Humanista_Peruano', '/wiki/Partido_Nacionalista_Peruano', '/wiki/Partido_Popular_Cristiano', '/wiki/Per%C3%BA_Patria_Segura', '/wiki/Per%C3%BA_Posible', '/wiki/Pedro_Pablo_Kuczynski', '/wiki/Restauraci%C3%B3n_Nacional_(Per%C3%BA)', '/wiki/Partido_Solidaridad_Nacional', '/wiki/Tierra_y_Libertad_(Per%C3%BA)', '/wiki/Uni%C3%B3n_por_el_Per%C3%BA']


In [46]:
wiki_pag.findAll("tr")

[<tr>
 <th style="background:Lavender; color:Black" width="250 px">Partidos políticos</th>
 <th style="background:Lavender; color:Black" width="250 px">Fecha de inscripción</th>
 <th style="background:Lavender; color:Black" width="250 px">Año de fundación</th>
 <th style="background:Lavender; color:Black" width="250 px">Sitio Web</th>
 </tr>, <tr>
 <td><a href="/wiki/Acci%C3%B3n_Popular" title="Acción Popular">Acción Popular</a></td>
 <td>4 de agosto de 2004</td>
 <td>1956</td>
 <td><a class="external text" href="http://www.accionpopular.com.pe" rel="nofollow">www.accionpopular.com.pe</a></td>
 </tr>, <tr>
 <td><a href="/wiki/Alianza_para_el_Progreso_(Per%C3%BA)" title="Alianza para el Progreso (Perú)">Alianza para el Progreso</a></td>
 <td>12 de febrero de 2008</td>
 <td>2001</td>
 <td><a class="external text" href="http://www.app-peru.org.pe" rel="nofollow">www.app-peru.org.pe</a></td>
 </tr>, <tr>
 <td>Democracia Directa</td>
 <td>11 de diciembre de 2013</td>
 <td></td>
 <td><a clas

## ¿Qué notas de los links que tienes hasta ahora?

Copia y pega los links que te ha dado la página web. ¿Has visto que no puedes ingresar a la página web? Esto pasa porque no está completo o le falta el url base. Nota que todas las páginas web de Wikipedia comienzan con `https://es.wikipedia.org/` y luego se incluye la dirección web exacta de la información que requerimos. Es esta segunda parte lo que tenemos. Felizmente que podemos usar lo que hemos aprendido de `strings` para solucionar esto.

### Ejercicio:

Entra a la página web de Acción Popular usando el la dirección de la lista previamente obtenida concatenándolo a la dirección base. Pista: No te olvides cómo se concatenan los strings `+`. Imprime el primer título y verifica que diga "Acción Popular".


In [47]:
url_base = "https://es.wikipedia.org"

pag_accion = requests.get(url_base + tabla_partidos[0])
pag_accion_obj = BeautifulSoup(pag_accion.text, "html.parser")
pag_accion_obj.h1.get_text()

'Acción Popular'

Hasta ahora todo va funcionando a la perección. Hemos obtenido nuestros links, sabemos cómo llegar a las páginas web mediante los links que ya tenemos en nuestra lista y, además, tenemos el código que nos permite extraer la información que necesitamos de cada página web. Es hora de unirlo todo para ver cómo funciona cuando lo juntamos.

In [48]:
for partido in tabla_partidos:
    pag_req = requests.get(url_base + partido)
    pag_req_obj = BeautifulSoup(pag_req.text, "html.parser")
    for item in pag_req_obj.find("table", {"class": "infobox"}).findAll("th", {"scope":"row"}):
        if item.get_text() in ["Presidente", "Ideología", "Congreso"]:
            print(item.find_next_sibling().get_text().strip())
    

Mesías Guevara Amasifuén
Liberalismo, Humanismo, Progresismo
5 / 130
César Acuña Peralta
Liberalismo
2 / 130
Keiko Fujimori
Fujimorismo[1]
Conservadurismo liberal
36 / 130
Alan García Pérez
Socialdemocracia
Socialismo democrático
Progresismo
4 / 130
Patricia Li Sotelo
Socialcristianismo
Yehude Simon Munaro
Liberalismo económico, Populismo
Nadine Heredia
Nacionalismo
Nacionalismo de izquierda
Socialismo
Latinoamericanismo[1]
35 / 130
Raúl Castro Stagnaro
Democracia cristiana
Socialcristianismo
Humanismo
5 / 130
Conservadurismo social
Alejandro Toledo
Liberalismo
Democracia liberal
Ecologismo
11 / 130
Alejandro Toledo
Alejandro Toledo
Alejandro Toledo
Fernando Belaúnde Terry
Humberto Lay Sun
Luis Castañeda Lossio
Conservadurismo
Liberalismo
Liberalismo conservador
7 / 130
Marco Arana Zegarra
Ecosocialismo,
Ecologismo radical,
Marxismo libertario,
Teología de la liberación
José Vega Antonio
Socioliberalismo
Mesías Guevara Amasifuén
Liberalismo, Humanismo, Progresismo
5 / 130
César Acuña P

Bien, hemos obtenido lo que queríamos. Sin embargo, nota que la información que tenemos está muy desordenada. No sabemos a qué corresponde qué dato. ¡Tratemos de ordernarlo un poco!

In [49]:
informacion_partidos = []
for partido in tabla_partidos:
    info = []
    pag_req = requests.get(url_base + partido)
    pag_req_obj = BeautifulSoup(pag_req.text, "html.parser")
    info.append(pag_req_obj.h1.get_text())
    for item in pag_req_obj.find("table", {"class": "infobox"}).findAll("th", {"scope":"row"}):
        if item.get_text() in ["Presidente", "Ideología", "Congreso"]:
            info.append(item.find_next_sibling().get_text().strip())
    informacion_partidos.append(info)

In [50]:
print(informacion_partidos)

[['Acción Popular', 'Mesías Guevara Amasifuén', 'Liberalismo, Humanismo, Progresismo', '5\xa0/\xa0130'], ['Alianza para el Progreso (Perú)', 'César Acuña Peralta', 'Liberalismo', '2\xa0/\xa0130'], ['Fuerza Popular', 'Keiko Fujimori', 'Fujimorismo[1]\nConservadurismo liberal', '36\xa0/\xa0130'], ['Ántero Flores-Aráoz'], ['Alianza Popular Revolucionaria Americana', 'Alan García Pérez', 'Socialdemocracia\nSocialismo democrático\nProgresismo', '4\xa0/\xa0130'], ['Partido Democrático Somos Perú', 'Patricia Li Sotelo', 'Socialcristianismo'], ['Partido Humanista Peruano', 'Yehude Simon Munaro', 'Liberalismo económico, Populismo'], ['Partido Nacionalista Peruano', 'Nadine Heredia', 'Nacionalismo\nNacionalismo de izquierda\nSocialismo\nLatinoamericanismo[1]', '35\xa0/\xa0130'], ['Partido Popular Cristiano', 'Raúl Castro Stagnaro', 'Democracia cristiana\nSocialcristianismo\nHumanismo', '5\xa0/\xa0130'], ['Perú Patria Segura', 'Conservadurismo social'], ['Perú Posible', 'Alejandro Toledo', 'Liber

## Pueden ver cuáles son los problemas de las páginas web solicitadas...

Aún la información no está de la manera que queremos. Noten que en algunos casos, i.e. Antero Flórez Aráoz y PPK, las direcciones de Wikipedia no son la de los partidos, sino la de los candidatos. En este caso, nuestro código debe identificar esto y poner como `missing` esta información. Por otro lado, hay ciertos partidos que no registran información, como número de Congresistas. 

## Primer problema: Queremos páginas de partidos y no páginas personales.

Una manera sencilla de verificar si estamos ingresando a una página web del partido o una del candidato, es ver si el título de la página web al que accedemos es el mismo título del partido que tenemos en la lista. Fíjense en el caso del partido "Peruanos por el Kambio": al momento de hacer click, entramos en la página de PPK. Esto no nos sirve. ¿Cómo podemos tener en cuenta esto para poner `missings` en nuestra base de datos? Intentemos que nuestro código chequee la coincidencia entre el título del link y el de la página web a la que estamos ingresando. Por el momento resolvamos este tema y luego veremos cómo implementarlo en nuestro código final.

In [51]:
nueva_tabla_partidos = []
for item in wiki_pag.findAll("tr"):
    try:
        nueva_tabla_partidos.append((item.find_next_sibling().find("td").a.get_text() ,item.find_next_sibling().find("td").a.attrs['href'])) ## Notar que aquí existen algunos None!
    except AttributeError:
        continue

print(nueva_tabla_partidos)

[('Acción Popular', '/wiki/Acci%C3%B3n_Popular'), ('Alianza para el Progreso', '/wiki/Alianza_para_el_Progreso_(Per%C3%BA)'), ('Fuerza Popular', '/wiki/Fuerza_2011'), ('Orden', '/wiki/%C3%81ntero_Flores-Ar%C3%A1oz'), ('Partido Aprista Peruano', '/wiki/Partido_Aprista_Peruano'), ('Partido Democrático Somos Perú', '/wiki/Partido_Democr%C3%A1tico_Somos_Per%C3%BA'), ('Partido Humanista Peruano', '/wiki/Partido_Humanista_Peruano'), ('Partido Nacionalista Peruano', '/wiki/Partido_Nacionalista_Peruano'), ('Partido Popular Cristiano', '/wiki/Partido_Popular_Cristiano'), ('Perú Patria Segura', '/wiki/Per%C3%BA_Patria_Segura'), ('Perú Posible', '/wiki/Per%C3%BA_Posible'), ('Peruanos Por el Kambio', '/wiki/Pedro_Pablo_Kuczynski'), ('Restauración Nacional', '/wiki/Restauraci%C3%B3n_Nacional_(Per%C3%BA)'), ('Solidaridad Nacional', '/wiki/Partido_Solidaridad_Nacional'), ('Tierra y Libertad', '/wiki/Tierra_y_Libertad_(Per%C3%BA)'), ('Unión por el Perú', '/wiki/Uni%C3%B3n_por_el_Per%C3%BA')]


In [52]:
for item in nueva_tabla_partidos:
    print(item[0], item[1])

Acción Popular /wiki/Acci%C3%B3n_Popular
Alianza para el Progreso /wiki/Alianza_para_el_Progreso_(Per%C3%BA)
Fuerza Popular /wiki/Fuerza_2011
Orden /wiki/%C3%81ntero_Flores-Ar%C3%A1oz
Partido Aprista Peruano /wiki/Partido_Aprista_Peruano
Partido Democrático Somos Perú /wiki/Partido_Democr%C3%A1tico_Somos_Per%C3%BA
Partido Humanista Peruano /wiki/Partido_Humanista_Peruano
Partido Nacionalista Peruano /wiki/Partido_Nacionalista_Peruano
Partido Popular Cristiano /wiki/Partido_Popular_Cristiano
Perú Patria Segura /wiki/Per%C3%BA_Patria_Segura
Perú Posible /wiki/Per%C3%BA_Posible
Peruanos Por el Kambio /wiki/Pedro_Pablo_Kuczynski
Restauración Nacional /wiki/Restauraci%C3%B3n_Nacional_(Per%C3%BA)
Solidaridad Nacional /wiki/Partido_Solidaridad_Nacional
Tierra y Libertad /wiki/Tierra_y_Libertad_(Per%C3%BA)
Unión por el Perú /wiki/Uni%C3%B3n_por_el_Per%C3%BA


In [53]:
for partido in nueva_tabla_partidos:
    pag_req = requests.get(url_base + partido[1])
    pag_req_obj = BeautifulSoup(pag_req.text, "html.parser")
    titulo_pag = pag_req_obj.find("th", {"class": "cabecera"}).get_text()
    if titulo_pag.lower() == partido[0].lower():
        print(partido[0], titulo_pag.lower(), "Sí")
    elif partido[0].lower() == "solidaridad nacional":
        print(partido[0], "Si")
    else:
        print(partido[0], titulo_pag.lower(), "No")

Acción Popular acción popular Sí
Alianza para el Progreso alianza para el progreso Sí
Fuerza Popular fuerza popular Sí
Orden ántero flores-aráoz esparza No
Partido Aprista Peruano partido aprista peruano Sí
Partido Democrático Somos Perú partido democrático somos perú Sí
Partido Humanista Peruano partido humanista peruano Sí
Partido Nacionalista Peruano partido nacionalista peruano Sí
Partido Popular Cristiano partido popular cristiano Sí
Perú Patria Segura perú patria segura Sí
Perú Posible perú posible Sí
Peruanos Por el Kambio pedro pablo kuczynski No
Restauración Nacional restauración nacional Sí
Solidaridad Nacional Si
Tierra y Libertad tierra y libertad Sí
Unión por el Perú unión por el perú Sí


In [54]:
informacion_partidos = []
for partido in nueva_tabla_partidos:
    presidente = None
    ideología = None
    congreso = None
    pag_req = requests.get(url_base + partido[1])
    pag_req_obj = BeautifulSoup(pag_req.text, "html.parser")
    info.append(pag_req_obj.h1.get_text())
    for item in pag_req_obj.find("table", {"class": "infobox"}).findAll("th", {"scope":"row"}):
        if item.get_text() == "Presidente":
            presidente = item.find_next_sibling().get_text().strip()
        elif item.get_text() == "Ideología":
            ideología = item.find_next_sibling().get_text().strip()
        elif item.get_text() == "Congreso":
            congreso = item.find_next_sibling().get_text().strip()
    informacion_partidos.append([partido[0], presidente, ideología, congreso])
print(informacion_partidos)

[['Acción Popular', 'Mesías Guevara Amasifuén', 'Liberalismo, Humanismo, Progresismo', '5\xa0/\xa0130'], ['Alianza para el Progreso', 'César Acuña Peralta', 'Liberalismo', '2\xa0/\xa0130'], ['Fuerza Popular', 'Keiko Fujimori', 'Fujimorismo[1]\nConservadurismo liberal', '36\xa0/\xa0130'], ['Orden', None, None, None], ['Partido Aprista Peruano', 'Alan García Pérez', 'Socialdemocracia\nSocialismo democrático\nProgresismo', '4\xa0/\xa0130'], ['Partido Democrático Somos Perú', 'Patricia Li Sotelo', 'Socialcristianismo', None], ['Partido Humanista Peruano', 'Yehude Simon Munaro', 'Liberalismo económico, Populismo', None], ['Partido Nacionalista Peruano', 'Nadine Heredia', 'Nacionalismo\nNacionalismo de izquierda\nSocialismo\nLatinoamericanismo[1]', '35\xa0/\xa0130'], ['Partido Popular Cristiano', 'Raúl Castro Stagnaro', 'Democracia cristiana\nSocialcristianismo\nHumanismo', '5\xa0/\xa0130'], ['Perú Patria Segura', None, 'Conservadurismo social', None], ['Perú Posible', 'Alejandro Toledo', 'L

## Uniendo todo + Limpiando data + guardando información

1. Generar una lista de solamente aqullos partidos que exploraremos. Eliminar aquellos que van a páginas personales de autores (PPK, Antero Flórez).
2. Extraer la información de las páginas de Wikipedia.
3. Limpiar la información.
4. Guardarla en un archivo CSV.


In [55]:
#1. Generando la lista que contenga los partidos depurados
part_depurados = []
for partido in nueva_tabla_partidos:
    pag_req = requests.get(url_base + partido[1])
    pag_req_obj = BeautifulSoup(pag_req.text, "html.parser")
    titulo_pag = pag_req_obj.find("th", {"class": "cabecera"}).get_text()
    if titulo_pag.lower() == partido[0].lower():
        part_depurados.append(partido)
    elif partido[0].lower() == "solidaridad nacional":
        part_depurados.append(partido)
    else:
        continue

print(part_depurados)

[('Acción Popular', '/wiki/Acci%C3%B3n_Popular'), ('Alianza para el Progreso', '/wiki/Alianza_para_el_Progreso_(Per%C3%BA)'), ('Fuerza Popular', '/wiki/Fuerza_2011'), ('Partido Aprista Peruano', '/wiki/Partido_Aprista_Peruano'), ('Partido Democrático Somos Perú', '/wiki/Partido_Democr%C3%A1tico_Somos_Per%C3%BA'), ('Partido Humanista Peruano', '/wiki/Partido_Humanista_Peruano'), ('Partido Nacionalista Peruano', '/wiki/Partido_Nacionalista_Peruano'), ('Partido Popular Cristiano', '/wiki/Partido_Popular_Cristiano'), ('Perú Patria Segura', '/wiki/Per%C3%BA_Patria_Segura'), ('Perú Posible', '/wiki/Per%C3%BA_Posible'), ('Restauración Nacional', '/wiki/Restauraci%C3%B3n_Nacional_(Per%C3%BA)'), ('Solidaridad Nacional', '/wiki/Partido_Solidaridad_Nacional'), ('Tierra y Libertad', '/wiki/Tierra_y_Libertad_(Per%C3%BA)'), ('Unión por el Perú', '/wiki/Uni%C3%B3n_por_el_Per%C3%BA')]


In [56]:
#2. Extrayendo información de páginas web de Wikipedia
informacion_partidos_dep = []
for partido in part_depurados:
    presidente = None
    ideología = None
    congreso = None
    pag_req = requests.get(url_base + partido[1])
    pag_req_obj = BeautifulSoup(pag_req.text, "html.parser")
    info.append(pag_req_obj.h1.get_text())
    for item in pag_req_obj.find("table", {"class": "infobox"}).findAll("th", {"scope":"row"}):
        if item.get_text() == "Presidente":
            presidente = item.find_next_sibling().get_text().strip()
        elif item.get_text() == "Ideología":
            ideología = item.find_next_sibling().get_text().strip()
        elif item.get_text() == "Congreso":
            congreso = item.find_next_sibling().get_text().strip()
    informacion_partidos_dep.append([partido[0], presidente, ideología, congreso])


Ahora es tiempo de limpiar un poco la información antes de incluirla en nuestro `CSV`. En particular haremos dos cosas. Primero quitaremos los espacios en la parte de ideología y los reemplazaremos por `,`. Lo segundo es modificar los asientos al congreso para que no muestren esos símbolos ni el total de escaños.

In [58]:
print(informacion_partidos_dep)

[['Acción Popular', 'Mesías Guevara Amasifuén', 'Liberalismo, Humanismo, Progresismo', '5\xa0/\xa0130'], ['Alianza para el Progreso', 'César Acuña Peralta', 'Liberalismo', '2\xa0/\xa0130'], ['Fuerza Popular', 'Keiko Fujimori', 'Fujimorismo[1]\nConservadurismo liberal', '36\xa0/\xa0130'], ['Partido Aprista Peruano', 'Alan García Pérez', 'Socialdemocracia\nSocialismo democrático\nProgresismo', '4\xa0/\xa0130'], ['Partido Democrático Somos Perú', 'Patricia Li Sotelo', 'Socialcristianismo', None], ['Partido Humanista Peruano', 'Yehude Simon Munaro', 'Liberalismo económico, Populismo', None], ['Partido Nacionalista Peruano', 'Nadine Heredia', 'Nacionalismo\nNacionalismo de izquierda\nSocialismo\nLatinoamericanismo[1]', '35\xa0/\xa0130'], ['Partido Popular Cristiano', 'Raúl Castro Stagnaro', 'Democracia cristiana\nSocialcristianismo\nHumanismo', '5\xa0/\xa0130'], ['Perú Patria Segura', None, 'Conservadurismo social', None], ['Perú Posible', 'Alejandro Toledo', 'Liberalismo\nDemocracia libera

In [59]:
for item in info_limpia:
    print(item[2], item[-1])
    

Liberalismo, Humanismo, Progresismo 5
Liberalismo 2
Fujimorismo[1], Conservadurismo liberal 36
Socialdemocracia, Socialismo democrático, Progresismo 4
Socialcristianismo None
Liberalismo económico, Populismo None
Nacionalismo, Nacionalismo de izquierda, Socialismo, Latinoamericanismo[1] 35
Democracia cristiana, Socialcristianismo, Humanismo 5
Conservadurismo social None
Liberalismo, Democracia liberal, Ecologismo 11
None None
Conservadurismo, Liberalismo, Liberalismo conservador 7
Ecosocialismo,, Ecologismo radical,, Marxismo libertario,, Teología de la liberación None
Socioliberalismo None


In [61]:
info_limpia = informacion_partidos_dep.copy()
for item in info_limpia:
    try:
        item[2] = item[2].replace("\n", ", ")
        item[-1] = item[-1].replace("\xa0/\xa0130", "")
    except AttributeError:
        continue
        

In [62]:
print(info_limpia)

[['Acción Popular', 'Mesías Guevara Amasifuén', 'Liberalismo, Humanismo, Progresismo', '5'], ['Alianza para el Progreso', 'César Acuña Peralta', 'Liberalismo', '2'], ['Fuerza Popular', 'Keiko Fujimori', 'Fujimorismo[1], Conservadurismo liberal', '36'], ['Partido Aprista Peruano', 'Alan García Pérez', 'Socialdemocracia, Socialismo democrático, Progresismo', '4'], ['Partido Democrático Somos Perú', 'Patricia Li Sotelo', 'Socialcristianismo', None], ['Partido Humanista Peruano', 'Yehude Simon Munaro', 'Liberalismo económico, Populismo', None], ['Partido Nacionalista Peruano', 'Nadine Heredia', 'Nacionalismo, Nacionalismo de izquierda, Socialismo, Latinoamericanismo[1]', '35'], ['Partido Popular Cristiano', 'Raúl Castro Stagnaro', 'Democracia cristiana, Socialcristianismo, Humanismo', '5'], ['Perú Patria Segura', None, 'Conservadurismo social', None], ['Perú Posible', 'Alejandro Toledo', 'Liberalismo, Democracia liberal, Ecologismo', '11'], ['Restauración Nacional', 'Humberto Lay Sun', Non

## Guardando la información

Ya tenemos la data limpia. Ahora lo que haremos es guardarla. Para esto tenemos que usar el módulo `CSV` de `Python`. Esto va a permitir que podamos generar y modificar de manera sencilla archivos separados por coma. Puedes aprender un poco más sobre cómo funciona este módulo [leyendo su documentación](https://docs.python.org/3.4/library/csv.html).
        
    

In [63]:
import csv
with open('info_partidos', 'w',  encoding = "utf-8") as csvfile:
    partidos = csv.writer(csvfile, dialect = "excel")
    for row in info_limpia:
        partidos.writerow(row)